## Cálculo de costos de transporte para cadenas productivas genéricas 

### Suposiciones
- Todas las etapas tienen un identificador único que se llama id
- La red es fija y la tabla se llama red
- La tabla de vértices se llama red_vertices_pgr
- tiene una columna que se llama costo
- tiene campos source y target
- todas las columnas de geometría se llaman geom

Primero todos los imports

In [82]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
from functools import reduce
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.transform import from_origin
import io
from subprocess import call

### Parámetros de la conexión
Define variables para hacer la conexión a la base de datos 

In [83]:
db = 'cadenas'
usr = 'postgres' 
psw = 'postgres' 
host = '192.168.18.22'

In [84]:
con = psycopg2.connect(database= db, user=usr, password=psw, host=host)
engine = create_engine('postgresql://{}:{}@{}:5432/{}'.format(usr,psw,host,db))

### Etapas de la cadena
Variable con los nombre de las tablas de etapas.

In [85]:
etapas = ['etapa1', 'etapa2', 'etapa3', 'etapa4' ]

### Relaciones entre nodos y etapas

Asignar a cada punto de cada etapa el nodo más cercano de la red. Se crean las tablas con las relaciones:`etapa_i_node`, cada tabla tiene dos columnas, el id de la tabla de etapas y el id de la tabla de nodos.


In [86]:
def node_relations(table, engine, connection):
    """Toma una tabla de puntos y le agrega una columna con el id del
       nodo más cercano de la red.
       Elimina la tabla de relaciones en caso de que exista.
       Regresa el dataframe con la relación.
    """
    drop_qry = """drop table if exists %(tabla)s_node""" % {"tabla": table}
    curs = connection.cursor()
    curs.execute(drop_qry)
    connection.commit()
    sql = """
            select f.id as id_%(tabla)s, (
              SELECT n.id
              FROM red_vertices_pgr As n
              ORDER BY f.geom <-> n.geom LIMIT 1
            )as closest_node
            from %(tabla)s f
          """ % {"tabla": table}
    try:
        df = pd.read_sql(sql, engine)
    except ValueError as e:
        print(e)
    try:
        df.to_sql(table + '_node', engine)
    except ValueError as e:
        print(e)
    return df

Crea una lista de tablas, con los identificadores del nodo más cercano y el id de la tabla original. 
NOTA: estas tablas NO TIENEN GEOMETRIAS

In [87]:
node_relations_list = []
for etapa in etapas:
    node_relations_list.append(node_relations(etapa, engine, con))

In [7]:
node_relations_list[-1].head()

,id_etapa4,closest_node
0,1,6156
1,2,1626
2,3,2827
3,4,5319
4,5,13025


Convierte cada tabla en GeoDataFrame y la agrega a la base de datos

In [88]:
etapas_gdfs = []
for etapa in etapas:
    sql = """select a.id, a.geom, b.closest_node
             from %(etapa)s a
             join %(etapa)s_node b
             on a.id = b.id_%(etapa)s""" % {"etapa":etapa}
    etapas_gdfs.append(gpd.GeoDataFrame.from_postgis(sql, con, geom_col='geom'))

In [89]:
etapas_gdfs[0].head()

,id,geom,closest_node
0,1737,POINT (645736.3108251573 1808481.380637101),16385
1,1,POINT (520212.1233636757 1896539.235540587),9104
2,2,POINT (551019.6914652064 1860672.211684213),7228
3,3,POINT (587381.6952086436 1652313.728707811),19701
4,4,POINT (552043.9602639768 1892579.33194623),24875


### Costos por etapa

La función stage_cost calcula el costo entre etapas sucesivas, para ello las tablas deben ir ordenadas 
dentro de esta función el usurio va a tener la posibilidad de seleccionar distintos valores asignados al costo, como: velocidad, tiempo, tipo de camino (coeficiente), valor de pendiente en grados y longitud. 

In [8]:
def stage_cost(source_table, target_table, cost_column):
    params = {'source': source_table, 'target': target_table, 'cost': cost_column }
    qry_str = """SELECT DISTINCT ON (start_vid)
                 start_vid as id_%(source)s, end_vid as id_%(target)s, agg_cost as costo_%(source)s_%(target)s
          FROM   (SELECT * FROM pgr_dijkstraCost(
              'select id, source, target, %(cost)s as cost from red',
              array(select distinct(s.closest_node) from (select e.*, r.closest_node
                                                        from %(source)s e
                                                        join %(source)s_node r
                                                        on e.id = r.id_%(source)s::int) as s),
              array(select distinct(t.closest_node) from (select e.*, r.closest_node
                                                        from %(target)s e
                                                        join %(target)s_node r
                                                        on e.id = r.id_%(target)s::int) as t),
                 directed:=false)
          ) as sub
          ORDER  BY start_vid, agg_cost asc""" % params
    try:
        df = pd.read_sql(qry_str, engine)
    except ValueError as e:
        print(e)
    return df

Para calcular la distancia entre dos etapas hacemos:

In [40]:
d0 = stage_cost("etapa1", "etapa2", "costo")
d0.head(10)

,id_etapa1,id_etapa2,costo_etapa1_etapa2
0,412,1851,268660.672500
1,459,1851,223833.071148
2,467,1851,233672.559784
3,638,11230,100905.641581
4,639,11230,105890.189821
5,714,1851,118419.029160
6,808,1851,32954.196776
7,809,1851,36366.841508
8,888,6662,74206.029518
9,889,6662,70608.639157


Y para calcular todas las distancias:

In [45]:
distancias = []
for i, etapa in enumerate(etapas): 
    if i < len(etapas)-1:
        stage = stage_cost(etapa,etapas[i+1], "costo")
        cost_col = list(stage.columns)[-1]
        stage.columns = ['start_' + str(i), 'end_' + str(i), cost_col]
        stage.to_sql('dist_' + etapa, engine, index=False, if_exists='replace')
        distancias.append(stage)

Al calcular las distancias obtenemos las columnas start_i, end_i, costo_etapa(origen)_etapa(destino)
donde: 
start_i = Tabla de origen
end_i = Tabla destino 
costo_etapa(origen)_etapa(destino) = es el valor asignado al costo acumulado entre etapas 

In [11]:
distancias[0].head()

,start_0,end_0,costo_etapa1_etapa2
0,412,1851,268660.672500
1,459,1851,223833.071148
2,467,1851,233672.559784
3,638,11230,100905.641581
4,639,11230,105890.189821


In [12]:
distancias[1].head()

,start_1,end_1,costo_etapa2_etapa3
0,1851,1626,14710.325903
1,4365,2827,5756.321023
2,4664,4252,94517.114344
3,5092,4252,13204.104910
4,5419,8350,11464.748789


In [13]:
distancias[2].head()

,start_2,end_2,costo_etapa3_etapa4
0,1506,2827,3669.320367
1,1626,5319,469.458292
2,2289,8350,10649.014814
3,2827,1506,3669.320367
4,4252,1506,53383.568474


Para poder calcular los rasters de costo, es necesario unir las tablas de costos por etapa y agregarlas a una sola tabla que tenga el tamaño de la primer etapa1. 

In [92]:
#c = pd.merge(distancias[0], distancias[1], left_on='end_0', right_on='start_1')
#d = pd.merge(c, distancias[2], left_on='end_1', right_on='start_2')
#d.head()
for i, distancia in enumerate(distancias):
    print(i)
    if i == 0:
        costos = pd.merge(distancia, distancias[1], left_on='end_0', right_on='start_1')
        print(list(costos.columns))
        
    elif i < len(distancias) - 1:
        costos = pd.merge(costos, distancias[i+1], left_on='end_' + str(i), right_on='start_' + str(i+1))
        print(list(costos.columns))

0
['start_0', 'end_0', 'costo_etapa1_etapa2', 'start_1', 'end_1', 'costo_etapa2_etapa3']
1
['start_0', 'end_0', 'costo_etapa1_etapa2', 'start_1', 'end_1', 'costo_etapa2_etapa3', 'start_2', 'end_2', 'costo_etapa3_etapa4']
2


In [94]:
costos.head()
#len(costos)

1674

Posteriormente se calcula la suma acumulada de costos y se conserva el id de inicio de la etapa1: 
   
   NOTA: Esta tabla NO TIENE GEOMETRÍAS

    costo_etapa1_etapa2 = raster1 
    costo_etapa1_etapa2 + costo_etapa2_etapa3 = raster2
    costo_etapa2_etapa3 + costo_etapa3_etapa4 = raster3
    

In [27]:
costos_acumulados = costos.iloc[:, costos.columns.str.contains('costo_')].cumsum(axis=1)
costos_acumulados = costos_acumulados.merge(costos.iloc[:,[0]], left_index=True, right_index=True)

In [28]:
costos_acumulados.head()

,costo_etapa1_etapa2,costo_etapa2_etapa3,costo_etapa3_etapa4,start_0
0,268660.672500,283370.998404,283840.456696,412
1,223833.071148,238543.397052,239012.855344,459
2,233672.559784,248382.885688,248852.343980,467
3,118419.029160,133129.355063,133598.813356,714
4,32954.196776,47664.522679,48133.980972,808


Para poder realizar la interpolación con GDAL, se asignan las geometrías de la tabla de la etapa1 
con el identificador start_0 y se seleccionan solo las columnas de interés

In [98]:
costos_acumulados = etapas_gdfs[0].merge(costos_acumulados, left_on = 'closest_node', right_on = 'start_0')
costos_acumulados.head()

,id,geom,closest_node,id_x,geom_x,closest_node_x,id_y,geom_y,closest_node_y,costo_etapa1_etapa2,costo_etapa2_etapa3,costo_etapa3_etapa4,start_0
0,1737,POINT (645736.3108251573 1808481.380637101),16385,1737,POINT (645736.3108251573 1808481.380637101),16385,1737,POINT (645736.3108251573 1808481.380637101),16385,88197.560033,89420.205764,120628.937040,16385
1,2,POINT (551019.6914652064 1860672.211684213),7228,2,POINT (551019.6914652064 1860672.211684213),7228,2,POINT (551019.6914652064 1860672.211684213),7228,43716.671817,56920.776727,110304.345201,7228
2,3,POINT (587381.6952086436 1652313.728707811),19701,3,POINT (587381.6952086436 1652313.728707811),19701,3,POINT (587381.6952086436 1652313.728707811),19701,74410.673774,76565.455704,129014.945168,19701
3,3,POINT (587381.6952086436 1652313.728707811),19701,3,POINT (587381.6952086436 1652313.728707811),19701,555,POINT (587025.9778533678 1653023.050466779),19701,74410.673774,76565.455704,129014.945168,19701
4,3,POINT (587381.6952086436 1652313.728707811),19701,555,POINT (587025.9778533678 1653023.050466779),19701,3,POINT (587381.6952086436 1652313.728707811),19701,74410.673774,76565.455704,129014.945168,19701


### Interpolación
Calcula los rasters de costo acumulados por etapa, cada raster contiene información del costo de transporte desde la etapa1 hasta la etapa a la que pertenece el raster. 

NOTA: Es en esta parte donde se hacen las recomendaciones y a diferencia del codigo pasado no es necesario hacer restas entre ellos. 

In [103]:
for columna in costos_acumulados:
    if columna.startswith('costo_'):
        write_me = costos_acumulados[['id', 'geom', columna]]
        write_me.columns = ['id', 'geom', 'costo']
        write_me.to_file(driver= 'ESRI Shapefile', filename= "idw/" + columna + '.shp')
        comando = ['gdal_grid', '-zfield', 'costo', '-l', columna, '-a',
           'invdist:power=2.0:smothing=0.0:radius1=0.0:radius2=0.0:angle=0.0:max_points=0:min_points=0:nodata=0.0',
           '-of', 'GTiff', 'idw/'+ columna + '.shp', 'idw/'+ columna+'.tif']
    try:
        call(comando)
        print('Si pude')
    except:
      print('La columna '+columna+' nostuvo')

Si pude
Si pude
Si pude
Si pude
Si pude
Si pude
Si pude
Si pude
Si pude


/home/eurekastein/anaconda3/envs/cadenas/lib/python3.6/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


Si pude


/home/eurekastein/anaconda3/envs/cadenas/lib/python3.6/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


Si pude


/home/eurekastein/anaconda3/envs/cadenas/lib/python3.6/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


Si pude
Si pude


## Ahora los polígonos


Se crean los poligonos para delimitar las zonas de influencias de cada etapa, siempre y cuando tenga más de tres puntos cerca


In [80]:
for j,(etapa, etapa_gdf) in enumerate(zip(etapas[1:],etapas_gdfs[1:])):
    drop_qry = """drop table if exists poligono_%(etapa)s""" % {'etapa':etapa}
    curs = con.cursor()
    curs.execute(drop_qry)
    con.commit()        
    create_sql = """create table poligono_%(etapa)s (id_%(etapa)s bigint, \
                    geom geometry(Polygon,32615))""" % {'etapa':etapa}
    curs.execute(create_sql)
    con.commit()    
    ids = etapa_gdf['closest_node'].unique()
    for i, id in enumerate(ids):
        point_sql = """select e.id::int4, st_x (st_geometryn(e.geom,1)) as x, st_y  (st_geometryn(e.geom,1)) as y 
            from 
            (select c.geom, d.end_%(num_etapa)s as id
            from 
            (select a.geom, b.closest_node 
            from %(etapa)s a
            join %(etapa)s_node b
            on a.id = b.id_%(etapa)s) as c
            join dist_%(etapa)s d
            on c.closest_node = d.start_%(num_etapa)s) as e 
            where id=%(de_quien)s
            """ % {'etapa':etapa, 'num_etapa':j+1, 'de_quien':id}
        point_gdf = pd.read_sql(point_sql, con)
        if point_gdf.shape[0] > 2:
            poly_sql = """
                insert into poligono_%(etapa)s
                select sub.id, sub.geom
                from
                (select %(de_quien)s as id, * from st_setsrid(pgr_pointsAsPolygon('select e.id::int4, st_x (st_geometryn(e.geom,1)) as x, st_y  (st_geometryn(e.geom,1)) as y 
                from 
                (select c.geom, d.end_%(num_etapa)s as id
                from 
                (select a.geom, b.closest_node 
                from %(etapa)s a
                join %(etapa)s_node b
                on a.id = b.id_%(etapa)s) as c
                join dist_%(etapa)s d
                on c.closest_node = d.start_%(num_etapa)s) as e 
                where id=%(de_quien)s'),32615)as geom)as sub 
            """ % {'etapa':etapa, 'num_etapa':j+1, 'de_quien':id}
            curs.execute(poly_sql)
            con.commit()
        #print(poly_sql)
        
        

DatabaseError: Execution failed on sql 'select e.id::int4, st_x (st_geometryn(e.geom,1)) as x, st_y  (st_geometryn(e.geom,1)) as y 
            from 
            (select c.geom, d.end_3 as id
            from 
            (select a.geom, b.closest_node 
            from etapa4 a
            join etapa4_node b
            on a.id = b.id_etapa4) as c
            join dist_etapa4 d
            on c.closest_node = d.start_3) as e 
            where id=6156
            ': no existe la relación «dist_etapa4»
LINE 9:             join dist_etapa4 d
                         ^


In [79]:
con.close()
con = psycopg2.connect(database= db, user=usr, password=psw, host=host)